# 02. 전처리 (Preprocessing)

SAPA 데이터를 전처리하고 성격 척도 점수를 계산합니다.

## 학습 목표
- Quality Control (QC): 응답 부족 및 Straight-lining 제외
- 역채점 처리 방법 이해
- Big Five, Ideology, Honesty-Humility 점수 계산


In [1]:
# 필요한 라이브러리 설치 (처음 한 번만 실행)
%pip install pandas numpy scipy -q



[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# 라이브러리 임포트
import pandas as pd
import numpy as np
from scipy import stats
import os

# 상위 폴더로 이동해서 데이터 접근
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')
print(f'작업 폴더: {os.getcwd()}')


작업 폴더: /Users/serinoh/serin-oh/safa


## 1. 데이터 로드


In [3]:
# SAPA 응답 데이터 로드
df = pd.read_csv('data/raw/sapa_data.csv')
keys = pd.read_csv('data/raw/superKey696.csv', index_col=0)

print(f'응답자 수: {len(df):,}')
print(f'변수 수: {len(df.columns)}')
print(f'채점 키: {keys.shape[0]}개 문항, {keys.shape[1]}개 척도')


응답자 수: 23,679
변수 수: 719
채점 키: 696개 문항, 131개 척도


In [4]:
# 성격 문항 컬럼 추출
item_cols = [col for col in df.columns if col.startswith('q_')]
print(f'성격 문항 수: {len(item_cols)}개')


성격 문항 수: 696개


## 2. Quality Control (QC)

다음 기준으로 응답자를 제외합니다:
1. **응답 부족**: 10개 미만의 문항에 응답한 경우
2. **Straight-lining**: 모든 응답이 동일한 값인 경우 (무응답 제외)


In [5]:
# QC 1: 응답 부족 (10개 미만)
responses_per_person = df[item_cols].notna().sum(axis=1)
insufficient_responses = responses_per_person < 10

print(f'응답 부족 (10개 미만): {insufficient_responses.sum():,}명')
print(f'평균 응답 문항 수: {responses_per_person.mean():.1f}개')


응답 부족 (10개 미만): 1명
평균 응답 문항 수: 86.1개


In [6]:
# QC 2: Straight-lining 감지
# 각 응답자의 응답값 중 고유값 개수 확인 (결측 제외)
def detect_straightlining(row):
    """응답이 모두 같은 값인지 확인 (결측 제외)"""
    non_missing = row.dropna()
    if len(non_missing) == 0:
        return False  # 모두 결측이면 straight-lining 아님
    return non_missing.nunique() == 1  # 고유값이 1개면 straight-lining

straightlining = df[item_cols].apply(detect_straightlining, axis=1)
print(f'Straight-lining: {straightlining.sum():,}명')


Straight-lining: 31명


In [7]:
# QC 통합: 제외할 응답자
exclude = insufficient_responses | straightlining
print(f'총 제외 인원: {exclude.sum():,}명')
print(f'유효 응답자 수: {(~exclude).sum():,}명')

# 유효한 응답자만 필터링
df_valid = df[~exclude].copy()
print(f'\n필터링 후 데이터 크기: {df_valid.shape}')


총 제외 인원: 32명
유효 응답자 수: 23,647명

필터링 후 데이터 크기: (23647, 719)


## 3. 척도 점수 계산 함수

채점 키를 사용하여 척도 점수를 계산합니다.
- `1`: 정채점
- `-1`: 역채점 (7 - 원점수, 6점 척도)
- `0`: 해당 없음


In [8]:
def calculate_scale_score(df, keys, scale_name):
    """
    채점 키를 사용해 척도 점수 계산
    
    Parameters:
    -----------
    df : DataFrame
        응답 데이터 (문항 컬럼 포함)
    keys : DataFrame
        채점 키 매트릭스 (index=문항명, columns=척도명)
    scale_name : str
        계산할 척도명 (예: 'NEO_E')
    
    Returns:
    --------
    Series
        각 응답자의 척도 점수
    """
    # 해당 척도에 속하는 문항 찾기
    scale_items = keys.index[keys[scale_name] != 0].tolist()
    weights = keys.loc[scale_items, scale_name]
    
    # 데이터에 있는 문항만 필터링
    available_items = [q for q in scale_items if q in df.columns]
    
    if not available_items:
        return pd.Series([np.nan] * len(df), index=df.index)
    
    # 역채점 적용 (6점 척도: 7 - 원점수)
    subset = df[available_items].copy()
    for item in available_items:
        if weights[item] == -1:
            subset[item] = 7 - subset[item]
    
    # 평균 계산 (결측 무시)
    return subset.mean(axis=1, skipna=True)


## 4. Big Five 점수 계산

NEO Big Five 성격 요인 점수를 계산합니다:
- Openness (개방성, NEO_O)
- Conscientiousness (성실성, NEO_C)
- Extraversion (외향성, NEO_E)
- Agreeableness (우호성, NEO_A)
- Neuroticism (신경증, NEO_N)


In [9]:
# 점수 저장용 DataFrame 초기화
scores = pd.DataFrame()
scores['RID'] = df_valid['RID'].values  # .values로 index 정보 제거

# Big Five 점수 계산
scores['NEO_Openness'] = calculate_scale_score(df_valid, keys, 'NEO_O').values
scores['NEO_Conscientiousness'] = calculate_scale_score(df_valid, keys, 'NEO_C').values
scores['NEO_Extraversion'] = calculate_scale_score(df_valid, keys, 'NEO_E').values
scores['NEO_Agreeableness'] = calculate_scale_score(df_valid, keys, 'NEO_A').values
scores['NEO_Neuroticism'] = calculate_scale_score(df_valid, keys, 'NEO_N').values

print('Big Five 점수 계산 완료')
print(f'점수 데이터 크기: {scores.shape}')


Big Five 점수 계산 완료
점수 데이터 크기: (23647, 6)


In [10]:
# Big Five 점수 요약 통계
big_five_cols = ['NEO_Openness', 'NEO_Conscientiousness', 'NEO_Extraversion', 
                 'NEO_Agreeableness', 'NEO_Neuroticism']

summary = pd.DataFrame({
    'N': scores[big_five_cols].notna().sum(),
    'Mean': scores[big_five_cols].mean(),
    'SD': scores[big_five_cols].std(),
    'Min': scores[big_five_cols].min(),
    'Max': scores[big_five_cols].max()
})
print('\n=== Big Five 점수 요약 ===')
print(summary.round(2))



=== Big Five 점수 요약 ===
                           N  Mean    SD  Min  Max
NEO_Openness           23399  4.31  0.83  1.0  6.0
NEO_Conscientiousness  23417  4.23  0.86  1.0  6.0
NEO_Extraversion       23357  3.87  0.92  1.0  6.0
NEO_Agreeableness      23437  4.23  0.82  1.0  6.0
NEO_Neuroticism        23371  3.35  0.98  1.0  6.0


## 5. Ideology (이념) 점수 계산

보수적 이념 점수는 다음 공식으로 계산합니다:
> **Ideology** = mean( z(MPQ_Traditionalism), z(NEO_Liberalism) × -1 )

- MPQ Traditionalism (MPQtr): 보수성 척도
- NEO Liberalism (NEOo6): 개방성 하위 요인, 역채점하여 보수성 지표로 변환


In [11]:
# Ideology 계산을 위한 하위 척도 점수
scores['MPQ_Traditionalism'] = calculate_scale_score(df_valid, keys, 'MPQtr').values
scores['NEO_Liberalism'] = calculate_scale_score(df_valid, keys, 'NEOo6').values

# z-score 계산 함수
def z_score(series):
    """표준화 (z-score)"""
    return (series - series.mean()) / series.std()

# Ideology 점수 계산 (z-score 후 평균)
z_mpq = z_score(scores['MPQ_Traditionalism'])
z_neo_lib = z_score(scores['NEO_Liberalism'])
scores['Ideology'] = (z_mpq + z_neo_lib * -1) / 2

print('Ideology 점수 계산 완료')
print(f'\nIdeology 요약:')
print(f'  N: {scores["Ideology"].notna().sum()}')
print(f'  Mean: {scores["Ideology"].mean():.3f}')
print(f'  SD: {scores["Ideology"].std():.3f}')


Ideology 점수 계산 완료

Ideology 요약:
  N: 11825
  Mean: 0.033
  SD: 0.969


## 6. Honesty-Humility (정직-겸손) 점수 계산

Honesty-Humility 점수는 다음 공식으로 계산합니다:
> **H-H** = mean( z(NEO_Morality), z(NEO_Modesty), z(HEXACO_H) )

- NEO Morality (NEOa2): 도덕성
- NEO Modesty (NEOa4): 겸손함
- HEXACO Honesty-Humility (HEXACO_H): 정직-겸손


In [12]:
# Honesty-Humility 계산을 위한 하위 척도 점수
scores['NEO_Morality'] = calculate_scale_score(df_valid, keys, 'NEOa2').values
scores['NEO_Modesty'] = calculate_scale_score(df_valid, keys, 'NEOa4').values
scores['HEXACO_H'] = calculate_scale_score(df_valid, keys, 'HEXACO_H').values

# Honesty-Humility 점수 계산 (z-score 후 평균)
z_morality = z_score(scores['NEO_Morality'])
z_modesty = z_score(scores['NEO_Modesty'])
z_hexaco_h = z_score(scores['HEXACO_H'])
scores['Honesty_Humility'] = (z_morality + z_modesty + z_hexaco_h) / 3

print('Honesty-Humility 점수 계산 완료')
print(f'\nHonesty-Humility 요약:')
print(f'  N: {scores["Honesty_Humility"].notna().sum()}')
print(f'  Mean: {scores["Honesty_Humility"].mean():.3f}')
print(f'  SD: {scores["Honesty_Humility"].std():.3f}')


Honesty-Humility 점수 계산 완료

Honesty-Humility 요약:
  N: 12828
  Mean: -0.003
  SD: 0.727


## 7. 최종 점수 요약

모든 척도 점수의 요약 통계를 확인합니다.


In [13]:
# 최종 점수 컬럼
final_score_cols = big_five_cols + ['Ideology', 'Honesty_Humility']

final_summary = pd.DataFrame({
    'N': scores[final_score_cols].notna().sum(),
    'Mean': scores[final_score_cols].mean(),
    'SD': scores[final_score_cols].std(),
    'Min': scores[final_score_cols].min(),
    'Max': scores[final_score_cols].max()
})

print('=== 최종 점수 요약 ===')
print(final_summary.round(3))


=== 최종 점수 요약 ===
                           N   Mean     SD    Min    Max
NEO_Openness           23399  4.308  0.827  1.000  6.000
NEO_Conscientiousness  23417  4.233  0.855  1.000  6.000
NEO_Extraversion       23357  3.868  0.922  1.000  6.000
NEO_Agreeableness      23437  4.225  0.816  1.000  6.000
NEO_Neuroticism        23371  3.355  0.979  1.000  6.000
Ideology               11825  0.033  0.969 -1.527  1.675
Honesty_Humility       12828 -0.003  0.727 -2.962  1.642


## 8. 결과 저장

계산된 점수를 CSV 파일로 저장합니다.


In [14]:
# processed 폴더가 없으면 생성
os.makedirs('data/processed', exist_ok=True)

# 점수 저장
output_path = 'data/processed/sapa_scores.csv'
scores.to_csv(output_path, index=False)

print(f'저장 완료: {output_path}')
print(f'점수 계산된 응답자 수: {len(scores):,}명')
print(f'저장된 척도: {len(final_score_cols)}개')
print(f'\n척도 목록:')
for col in final_score_cols:
    print(f'  - {col}')


저장 완료: data/processed/sapa_scores.csv
점수 계산된 응답자 수: 23,647명
저장된 척도: 7개

척도 목록:
  - NEO_Openness
  - NEO_Conscientiousness
  - NEO_Extraversion
  - NEO_Agreeableness
  - NEO_Neuroticism
  - Ideology
  - Honesty_Humility


## 9. QC 리포트 요약

전처리 과정에서 제외된 응답자 정보를 요약합니다.


In [15]:
qc_summary = {
    '원본 응답자 수': len(df),
    '응답 부족 제외 (10개 미만)': insufficient_responses.sum(),
    'Straight-lining 제외': straightlining.sum(),
    '총 제외 인원': exclude.sum(),
    '최종 유효 응답자 수': len(df_valid),
    '유효 응답자 비율': f"{(~exclude).sum() / len(df) * 100:.1f}%"
}

print('=== QC 리포트 ===')
for key, value in qc_summary.items():
    if isinstance(value, int):
        print(f'{key}: {value:,}명')
    else:
        print(f'{key}: {value}')


=== QC 리포트 ===
원본 응답자 수: 23,679명
응답 부족 제외 (10개 미만): 1
Straight-lining 제외: 31
총 제외 인원: 32
최종 유효 응답자 수: 23,647명
유효 응답자 비율: 99.9%
